# LIBRAIRIES

In [1]:
import datetime
import re
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException

from bs4 import BeautifulSoup
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError
import matplotlib.pyplot as plt
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords

from yellowbrick.cluster import KElbowVisualizer
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV

import unidecode

In [2]:
df_original=pd.read_csv("salary_indeed.csv")

In [3]:
df_original.shape

(4884, 9)

# Analyze optimization memory

In [4]:
df_original.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4884 entries, 0 to 4883
Data columns (total 9 columns):
_id                  4884 non-null object
Title                4884 non-null object
Company              4879 non-null object
Location             4884 non-null object
Salary               744 non-null object
Description          4884 non-null object
Date                 4376 non-null object
Job_Search           4884 non-null object
Department_Search    4884 non-null object
dtypes: object(9)
memory usage: 24.0 MB


In [5]:
df_original.describe()

,_id,Title,Company,Location,Salary,Description,Date,Job_Search,Department_Search
count,4884,4884,4879,4884,744,4884,4376,4884,4884
unique,4884,4072,1879,135,231,4818,50,2,5
top,p_381926a44a5c8056,Data Engineer,TALENTS RH,Paris (75),35 000 € - 45 000 € par an,"TALENTS RH, société de recrutement spécialisée...",07/09/2019,Développeur,75
freq,1,25,66,1972,31,4,918,3304,2678


In [6]:
df_original.isna().sum()

_id                     0
Title                   0
Company                 5
Location                0
Salary               4140
Description             0
Date                  508
Job_Search              0
Department_Search       0
dtype: int64

In [7]:
df_original.sample(5)

,_id,Title,Company,Location,Salary,Description,Date,Job_Search,Department_Search
1044,p_3aab44c61d269eec,Ingénieur data scientist – Markplace référente,Konekt,Paris 10e (75),50 000 € - 55 000 € par an,"Description de l'emploi Mon client, LA marketp...",07/09/2019,Data,75
1525,p_5812d62422a1f0f8,Développeur PHP (H/F) à Rillieux-la-Pape (69),XEFI,Rillieux-la-Pape (69),NaN,"Qui sommes-nous ? Créé en 1997, le groupe XEFI...",07/09/2019,Développeur,Rhône
4830,pj_e7009a14a12c6ff3,Développeur Frontend Javascript H/F,Approach People - MP,Paris 1er (75),45 000 € - 60 000 € par an,Développeur Frontend Javascript H/F Entreprise...,NaN,Développeur,75
3281,p_c2d5433021a6148d,Software Engineer,Fifty-Five,Paris (75),NaN,fifty-five accompagne les entreprises dans l’e...,09/06/2019,Développeur,75
825,p_2dde0295ad37faac,"Devops Network Engineer, Engineering - Infrast...",Criteo,Paris (75),NaN,"Who we are At Criteo, we connect 1.5 billion a...",14/09/2019,Développeur,75


In [8]:
converted_obj = pd.DataFrame()
for col in df_original.columns:
    num_unique_values = len(df_original[col].unique())
    num_total_values = len(df_original[col])
    if num_unique_values / num_total_values < 0.5:
        converted_obj.loc[:,col] = df_original[col].astype('category')
    else:
        converted_obj.loc[:,col] = df_original[col]

In [9]:
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)

In [10]:
print(mem_usage(df_original))
print(mem_usage(converted_obj))

35.30 MB
33.71 MB


In [11]:
compare_obj = pd.concat([df_original.dtypes,converted_obj.dtypes],axis=1)
compare_obj.columns = ['before','after']
compare_obj.apply(pd.Series.value_counts)

,before,after
object,9.0,3
category,NaN,1
category,NaN,1
category,NaN,1
category,NaN,1
category,NaN,1
category,NaN,1


In [12]:
converted_obj.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4884 entries, 0 to 4883
Data columns (total 9 columns):
_id                  4884 non-null object
Title                4884 non-null object
Company              4879 non-null category
Location             4884 non-null category
Salary               744 non-null category
Description          4884 non-null object
Date                 4376 non-null category
Job_Search           4884 non-null category
Department_Search    4884 non-null category
dtypes: category(6), object(3)
memory usage: 33.7 MB


We don't need to change dtypes (little change)

# PRE-PROCESSING

# Cleaning title

In [13]:
def clean_title(title):
    title = word_tokenize(title)
    title = [word.lower() for word in title if (word.isalpha() or word.isdigit()) and len(word)>1]
    title = [word for word in title if word not in stop_words_bi]
    return " ".join(title)

In [14]:
stop_words_bi = stopwords.words('french') + stopwords.words('english')

In [15]:
df_original["clean_title"]=df_original.Title.apply(clean_title)

In [16]:
df_original.clean_title

0           développeuse développeur stack elastic search
1                                   développeur front end
2                                 data analyst alternance
3       internship position evaluation aircraft data t...
4                                   développeurs java jee
                              ...                        
4879                                 digital data analyst
4880                   développeur front end react native
4881                                     ingénieur devops
4882                  analyste développeur sap full stack
4883                                          développeur
Name: clean_title, Length: 4884, dtype: object

In [17]:
df_original.clean_title.describe()

count            4884
unique           2879
top       développeur
freq              215
Name: clean_title, dtype: object

In [18]:
list(df_original.clean_title.unique())

['développeuse développeur stack elastic search',
 'développeur front end',
 'data analyst alternance',
 'internship position evaluation aircraft data traffic cost',
 'développeurs java jee',
 'lead developpeur python',
 'consultant confirmé bi data intelligence',
 'consultant data',
 'ingénieur développeur java',
 'devops spécialiste openstack',
 'développeur web bases données',
 'data engineer java',
 'devops junior data management middleware',
 'ingenieur developpeur firmware',
 'data engineer santé',
 'devops',
 'développeur angular',
 'data scientist',
 'cloud data architect manager secteur santé',
 'business developer btob data intelligence',
 'data manager experimente',
 'développeurs frontend angularjs ios android devops',
 'analyste developpeur web',
 'staff data engineer',
 'développeur php consultant lamp',
 'ingénieur réseaux sécurité devops',
 'alternance développeur applications nouvelles technologies',
 'développeur calculateur risques contrepartie',
 'software engineer'

# Creating column with key_words title

In [19]:
df_original["Title_data"]=df_original.Title.str.lower().str.contains(r'data',regex=True).astype(int)

df_original["Title_data_analyst"]=df_original.Title.str.lower().str.contains(r'data anal[iy]',regex=True).astype(int)

df_original["Title_data_scientist"]=df_original.Title.str.lower().str.contains(r'data scientist',regex=True).astype(int)

df_original["Title_data_engineer"]=df_original.Title.str.lower().str.contains(r'data engin',regex=True).astype(int)

df_original["Title_data_architect"]=df_original.Title.str.lower().str.contains(r'data architect',regex=True).astype(int)

df_original["Title_ingenieur"]=df_original.Title.str.lower().str.contains(r'ing[ée]n|engin',regex=True).astype(int)

df_original["Title_big_data"]=df_original.Title.str.lower().str.contains(r'big\s?data',regex=True).astype(int)

df_original["Title_developpeur"]=df_original.Title.str.lower().str.contains(r'd[ée]veloppeu[r]?(se)?[s]?|developer',regex=True).astype(int)

df_original["Title_dev_frontend"]=df_original.Title.str.lower().str.contains(r'front\s?(end)?',regex=True).astype(int)

df_original["Title_dev_web"]=df_original.Title.str.lower().str.contains(r'php|web|html',regex=True).astype(int)

df_original["Title_dev_mobile"]=df_original.Title.str.lower().str.contains(r'mobile|android|ios|swift',regex=True).astype(int)

df_original["Title_dev_back_end"]=df_original.Title.str.lower().str.contains(r'back\s?(end)?',regex=True).astype(int)

df_original["Title_dev_fullstack"]=df_original.Title.str.lower().str.contains(r'full\sstack',regex=True).astype(int)

df_original["Title_devops"]=df_original.Title.str.lower().str.contains(r'devops',regex=True).astype(int)

df_original["Title_software_eng"]=df_original.Title.str.lower().str.contains(r'software engineer',regex=True).astype(int)

df_original["Title_junior"]=df_original.Title.str.lower().str.contains(r'junior',regex=True).astype(int)

df_original["Title_manager"]=df_original.Title.str.lower().str.contains(r'manager',regex=True).astype(int)

df_original["Title_BI"]=df_original.Title.str.lower().str.contains(r'business intel|\bbi\b',regex=True).astype(int)

df_original["Title_confirme"]=df_original.Title.str.lower().str.contains(r'confirm',regex=True).astype(int)

df_original["Title_lead"]=df_original.Title.str.lower().str.contains(r'lead',regex=True).astype(int)

df_original["Title_senior"]=df_original.Title.str.lower().str.contains(r's[ée]nior',regex=True).astype(int)

df_original["Title_consultant"]=df_original.Title.str.lower().str.contains(r'consult',regex=True).astype(int)

df_original["Title_directeur"]=df_original.Title.str.lower().str.contains(r'dir[ée]cte',regex=True).astype(int)

/Users/Ivan/anaconda3/lib/python3.7/site-packages/pandas/core/strings.py:1843: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [20]:
#df_original[df_original.Title.str.lower().str.contains(r'manageu',regex=True)]

# DATA RECOVERY

## EXPERIENCES

In [21]:
#import re
#import numpy as np
#import pandas as pd
#from nltk import word_tokenize
#from nltk.corpus import stopwords
#import unidecode

#stopwords
stop_words = stopwords.words('french')

def preprocessing_xp(text, stopwords):
    text = text.lower()
    text = re.sub(r'bac\s?\+\s?\d\/?\d?', ' ', text)
    text = re.sub(r'[^\w|\s|+]', ' ', text) #take off punctuation sign, keep space and plus sign
    text = re.sub(r'[_|\|]', ' ', text) #take off _ and |
    text = unidecode.unidecode(text) #take off accent on letters
    text = re.sub(r'\b[abd-qs-z]\b', ' ', text) #for single letters : keep only C and R (programming languages)
    text = re.sub(r'\d{3,}|[2-9][0-9]|\b00+\b', ' ', text) #take off number from 3 character (100 and more), take off number between 20-99, and all number with only 00, 000, ...
    
    # STOPWORDS
    tokenized_words = word_tokenize(text) #put words in a list without space
    tokenized_words = [word for word in tokenized_words if word not in stopwords] 

    return " ".join(tokenized_words) #put the list back as phrases

#call preprocess skills functions
df_original["description_clean"] = df_original.Description.apply(preprocessing_xp, args=(stop_words,))

#keep 3 words before "annee(s)" --- annee(s) experience(s)
df_original["experience_left"] = df_original.description_clean\
                                    .apply(lambda x: re.findall(r'((?:\S+\s+){0,3}\ban(?:nee)?s?\b experience)', x))

#keep everything between : experience(s) annee(s), limit 150 characters
df_original["experience_right"] = df_original.description_clean\
                                    .apply(lambda x: re.findall(r'experience.{,150}?\ban(?:nee)?s?\b', x))

#create list of numbers (experience_left)
df_original["experience_left_list"] = df_original.experience_left\
                                        .apply(lambda x: re.findall(r'\b\d+\b', repr(x)))

#create list of numbers (experience_right)
df_original["experience_right_list"] = df_original.experience_right\
                                            .apply(lambda x: re.findall(r'\b\d+\b', repr(x)))

#link left-right columns
df_original["experience_total"] = (df_original["experience_left_list"]
                                        + df_original["experience_right_list"])

#b convert column in float and if empty put NaN in
df_original["experience_clean"] = df_original.experience_total\
                                    .apply(lambda y: np.array(y).astype(np.float)
                                           if len(y)>0 else np.nan)

# create min and max experiences columns
df_original["experience_min"] = df_original["experience_clean"].apply(np.amin)
df_original["experience_max"] = df_original["experience_clean"].apply(np.amax)

df_original = df_original.drop(['description_clean', 'experience_left',
                                    'experience_right', 'experience_left_list',
                                    'experience_right_list', 'experience_total',
                                    'experience_clean'], axis=1)

## QUALIFICATIONS

In [22]:
df_original["BTS"] = df_original.Description.str.lower().str.contains(r'\bbts\b', regex=True).astype(int)

df_original["DEUG"] = df_original.Description.str.lower().str.contains(r'\bdeug\b|\bniveaux?iii+\b|\bbac[\s\S]?[+]?[\s\S]2\b', regex=True).astype(int)

df_original["DUT"] = df_original.Description.str.lower().str.contains(r'\bdut\b', regex=True).astype(int)

df_original["IUT"] =df_original.Description.str.lower().str.contains(r'\biut\b', regex=True).astype(int)

df_original["Licence"] = df_original.Description.str.lower().str.contains(r'\blicence\b|\bniveaux?ii+\b|\bbac[\s\S]?[+]?[\s\S]3\b|\b\(?bac\s?\+\s?2\s?[\/àeo][tu]?\s?(bac)?\s?\+?\s?5\)?\b|(\b\(?bac\s?\+\s?2\s?[\/àeo][tu]?\s?(bac)?\s?\+?\s?3\)?\b)', regex=True).astype(int)

df_original["Master"] = df_original.Description.str.lower().str.contains(r'\bmaster\b|\bniveaux?i+\b|\bbac[\s\S]?[+]?[\s\S]5\b|\bmba\b|(\bbac\s?\+\s?4\s?[\/à]\s?(bac)?\+?5\b)|(\bbac\s?\+\s?4\s?(ou)\s?(bac)?\s?\+?\s?5\b)', regex=True).astype(int)

df_original["Doctorat"] = df_original.Description.str.lower().str.contains(r'\bdoctorat\b|\bbac[\s\S]?[+]?[\s\S]8\b', regex=True).astype(int)

df_original["Ingenieur"] = df_original.Description.str.lower().str.contains(r'\b[eé]cole\s?d?.?\s?ing[ée]nieur[e]?\b', regex=True).astype(int)

## CONTRACTS

In [23]:
#import pandas as pd
#import numpy as np
#import matplotlib.pyplot as plt
#import nltk
#from nltk import word_tokenize
#from nltk.corpus import stopwords
#import re

def type_de_contrat(row):
    title = row["Title"].lower()
    desc = row["Description"].lower()
    #search for CDI in title and desc
    if bool(re.search('cdi',title)):
        row["CDI"]=1
    elif bool(re.search('cdi',desc)) and not bool(re.search('stage',title)):
        row["CDI"]=1
    else:
        row["CDI"]=0
    #search for stage in title and desc
    if bool(re.search(r'stag|intern',title)):
        row["Stage"]=1 
    elif bool(re.search(r'type\sd(\\?\'emploi\s?|e\scontrat):?.{0,26}stage',desc)) and not bool(re.search('stage',title)):
        row["Stage"]=1
    else:
        row["Stage"]=0
    #search for freelance in title and desc
    if bool(re.search('freelance',title)):
        row["Freelance"]=1
    elif bool(re.search(r'type\sd(\\?\'emploi\s?|e\scontrat):?.{0,26}freelance',desc)) and not bool(re.search('freelance',title)):
        row["Freelance"]=1
    else:
        row["Freelance"]=0
    #search for CDD in title and desc
    if bool(re.search('cdd',title)):
        row["CDD"]=1
    elif bool(re.search(r'type\sd(\\?\'emploi\s?|e\scontrat):?.{0,26}cdd',desc)) and not bool(re.search('cdd',title)):
        row["CDD"]=1 
    else:
        row["CDD"]=0
    #si stage alors on met en zero les autres contrats
    if row["Stage"]==1:
        row["CDD"]=0
        row["Freelance"]=0
        row["CDI"]=0
        
    return row

#apply function on dataframe
df_original= df_original.apply(type_de_contrat, axis=1)

#get no restriction on column table vizualization on jupyter notebook (set to 100)

pd.set_option('display.max_columns', 100)

## COMPETENCES & KEYS WORDS

1. Mettre en minuscule
2. Remplacer les ponctuations (sauf '+') par des espaces : `[^\w|\s|+]` mais aussi les '|' et '\_' : `[_|\|]`
3. Remplacer les lettres accentuées par des lettres sans accents
4. Remplacer les lettres seules (sauf les lettres c et r (langages de programmation)) par des espaces : `\b[abd-qs-z]\b`
5. Remplacer les nombres qui ont 2 chiffres ou plus par des espaces : `\d{2,}`
6. Splitter la chaîne de caractères en une liste de mots
7. Créer une nouvelle liste sans les stopwords
8. Retourne la liste de mots en chaîne de caractères

In [24]:
def preprocessing_skills(text, stopwords, prefix=''):
    if prefix:
        prefix = prefix.lower() + '_'

    text = text.lower()
    text = re.sub(r'[^\w|\s|+]', ' ', text)
    text = re.sub(r'[_|\|]', ' ', text)
    text = unidecode.unidecode(text)
    text = re.sub(r'\b[abd-qs-z]\b', ' ', text)
    text = re.sub(r'\d{2,}', ' ', text)

    # STOPWORDS
    tokenized_words = word_tokenize(text)
    tokenized_words = [prefix + word for word in tokenized_words if word not in stopwords]

    return " ".join(tokenized_words)

#call preprocess skills functions
df_original["description_clean"] = df_original.Description.apply(preprocessing_skills, args=(stop_words,))


df_original['Python'] =  df_original.description_clean.str.contains(r'python|pyhton|pyton', regex=True).astype(int)
df_original['Java'] = df_original.description_clean.str.contains(r'java', regex=True).astype(int)
df_original['Machine Learning'] = df_original.description_clean.str.contains(r'machine learning|scikit learn|sklearn', regex=True).astype(int)
df_original['Deep Learning'] = df_original.description_clean.str.contains(r'deep learning|keras|tensor\s?flow|pytorch|theano', regex=True).astype(int)
df_original['Javascript'] = df_original.description_clean.str.contains(r'javascript|angular|react|js|jquery', regex=True).astype(int)
df_original['Swift'] = df_original.description_clean.str.contains(r'swift|ios', regex=True).astype(int)
df_original['NoSQL'] = df_original.description_clean.str.contains(r'mongodb|no sql|nosql', regex=True).astype(int)
df_original['SQL'] = df_original.description_clean.str.contains(r'\bsql\b|mysql|postgresql', regex=True).astype(int)
df_original['Agile'] = df_original.description_clean.str.contains(r'agile|scrum', regex=True).astype(int)
df_original['J2EE'] = df_original.description_clean.str.contains(r'j2ee|jee', regex=True).astype(int)
df_original['Ruby'] = df_original.description_clean.str.contains(r'ruby|rails', regex=True).astype(int)

df_original['HTML_CSS'] = df_original.description_clean.str.contains(r'html|css', regex=True).astype(int)

df_original['Php'] = df_original.description_clean.str.contains(r'php|symfony', regex=True).astype(int)
df_original['Big_Data'] = df_original.description_clean.str.contains(r'bigdata|big data|spark|hadoop', regex=True).astype(int)
df_original['Scala'] = df_original.description_clean.str.contains(r'scala', regex=True).astype(int)
df_original['Back_end'] = df_original.description_clean.str.contains(r'back end|backend', regex=True).astype(int)
df_original['Git'] = df_original.description_clean.str.contains(r'git|github', regex=True).astype(int)
df_original['R'] = df_original.description_clean.str.contains(r'\br\b', regex=True).astype(int)
df_original['PowerBI'] = df_original.description_clean.str.contains(r'powerbi|power bi', regex=True).astype(int)
df_original['Cloud'] = df_original.description_clean.str.contains(r'googlecloud|google cloud|aws|azure|\bcloud\b', regex=True).astype(int)
df_original['C_C++'] = df_original.description_clean.str.contains(r'\bc\b|\bc\+\+\b', regex=True).astype(int)
df_original['Docker'] = df_original.description_clean.str.contains(r'\bdocker\b', regex=True).astype(int)
df_original['ABAP'] = df_original.description_clean.str.contains(r'\bsap\b|\babap\b', regex=True).astype(int)
df_original['WLang'] = df_original.description_clean.str.contains(r'windev|webdev', regex=True).astype(int)

In [25]:
df_original.shape

(4884, 72)

# DATA ORGANIZATION

In [26]:
#import pandas as pd
#import numpy as np
#from yellowbrick.cluster import KElbowVisualizer
#from sklearn.cluster import KMeans
#from sklearn.cluster import DBSCAN
#from sklearn.preprocessing import StandardScaler
#from sklearn.model_selection import train_test_split
#from sklearn.decomposition import PCA
#from sklearn.model_selection import GridSearchCV
#import nltk
#from nltk import word_tokenize
#from nltk.corpus import stopwords
#import matplotlib.pyplot as plt

# Localisation

## Split_city function creation

On sépare la ville et le cp. On remplace les differents arrondisements de Paris et Lyon par la ville. On remplis les valeurs manquantes par le départment de recherche

In [27]:
def split_city(row):
    location = row["Location"]
    if "Paris" in location or location == "France":
        city = "Paris"
        code_postal = "75"
    elif "Lyon" in location:
        city = "Lyon"
        code_postal = "69"
    elif "(" in location:
        split = location.split("(")
        city = split[0]
        code_postal = split[1]
    else:
        city = row["Department_Search"]
        code_postal="None"
    
    row["city"] = city
    row["cp"] = code_postal.replace(")","")
    return row

df_original = df_original.apply(split_city, axis=1)

## Split_salary function creation (salary min, max, mean)

Création des colonnes salaire_max, salaire_min et type de salaire. On transforme en annuelle seulement les valeurs en mois qui sont supérieur à 1500 smic brut + création de la colonne type de salaire (annuelle, mensuelle, hebdomadaire et par heure)

In [28]:
def split_salary(row):
    salary = row["Salary"]
    if "-" in salary:
        split = salary.split("-")
        salary_min = split[0]
        salary_max = split[1]
    else:
        salary_min = salary
        salary_max = salary
    
    row["salary_min"] = salary_min.replace("€","")\
                                  .replace("par an","")\
                                  .replace("par mois","")\
                                  .replace("par semaine","")\
                                  .replace("par jour","")\
                                  .replace("par heure","")\
                                  .replace("\xa0","")
    row["salary_max"] = salary_max.replace("€","")\
                                  .replace("par an","")\
                                  .replace("par mois","")\
                                  .replace("par semaine","")\
                                  .replace("par jour","")\
                                  .replace("par heure","")\
                                  .replace("\xa0","")
    
    if "an" in row["Salary"]:
        row["salary_period"] = "year"

    if "mois" in row["Salary"]:
        if float(row["salary_min"]) < 1500:
            row["salary_min"] = float(row["salary_min"])
            row["salary_max"] = float(row["salary_max"])
            row["salary_period"] = "month"
        else:
            row["salary_min"] = float(row["salary_min"])*12
            row["salary_max"] = float(row["salary_max"])*12
            row["salary_period"] = "year"
    
    if "semaine" in row["Salary"]:
        row["salary_min"] = float(row["salary_min"])
        row["salary_max"] = float(row["salary_max"])
        row["salary_period"] = "week"

    if "jour" in row["Salary"]:
        row["salary_min"] = float(row["salary_min"])*5
        row["salary_max"] = float(row["salary_max"])*5
        row["salary_period"] = "week"

    if "heure" in row["Salary"]:
        row["salary_min"] = float(row["salary_min"].replace(",","."))*35
        row["salary_max"] = float(row["salary_max"].replace(",","."))*35
        row["salary_period"] = "week"
        
    return row

# Keep only rows with salary
df_salary = df_original[~df_original.Salary.isna()]

df_salary = df_salary.apply(split_salary, axis=1)
df_salary["salary_min"]=pd.to_numeric(df_salary["salary_min"],'coerce', downcast="float")
df_salary["salary_max"]=pd.to_numeric(df_salary["salary_max"],'coerce',downcast="float")
df_salary["salary_mean"] = (df_salary["salary_min"]+df_salary["salary_max"])/2

In [29]:
print("Nombre de lignes avec salaire en années : {}".format(df_salary[df_salary["salary_period"]=="year"].shape[0]))
print("Nombre de lignes avec salaire en mois : {}".format(df_salary[df_salary["salary_period"]=="month"].shape[0]))
print("Nombre de lignes avec salaire en semaine : {}".format(df_salary[df_salary["salary_period"]=="week"].shape[0]))

Nombre de lignes avec salaire en années : 675
Nombre de lignes avec salaire en mois : 30
Nombre de lignes avec salaire en semaine : 39


In [30]:
df_salary[df_salary["salary_period"]=="month"] # visualisation lignes saliare mensuelle

,_id,Title,Company,Location,Salary,Description,Date,Job_Search,Department_Search,clean_title,Title_data,Title_data_analyst,Title_data_scientist,Title_data_engineer,Title_data_architect,Title_ingenieur,Title_big_data,Title_developpeur,Title_dev_frontend,Title_dev_web,Title_dev_mobile,Title_dev_back_end,Title_dev_fullstack,Title_devops,Title_software_eng,Title_junior,Title_manager,Title_BI,Title_confirme,Title_lead,Title_senior,Title_consultant,Title_directeur,experience_min,experience_max,BTS,DEUG,DUT,IUT,Licence,Master,Doctorat,Ingenieur,CDI,Stage,Freelance,CDD,description_clean,Python,Java,Machine Learning,Deep Learning,Javascript,Swift,NoSQL,SQL,Agile,J2EE,Ruby,HTML_CSS,Php,Big_Data,Scala,Back_end,Git,R,PowerBI,Cloud,C_C++,Docker,ABAP,WLang,city,cp,salary_min,salary_max,salary_period,salary_mean
339,p_11932114a525d757,STAGE Chef de projet - Customer Data Managemen...,Danone,Limonest (69),1 120 € par mois,Danone recherche un(e) STAGE Chef de projet - ...,06/10/2019,Data,Rhône,stage chef projet customer data management moi...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0,0,0,1,0,0,0,1,0,0,danone recherche stage chef projet customer da...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,Limonest,69,1120.0,1120.0,month,1120.0
358,p_13477b94e2936148,Stage software et Data Engineer,Outmind,Paris 8e (75),600 € - 1 200 € par mois,À propos Outmind est une jeune startup créant ...,07/09/2019,Data,75,stage software data engineer,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,6.0,0,0,0,0,0,1,0,0,0,1,0,0,propos outmind jeune startup creant moteur rec...,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,Paris,75,600.0,1200.0,month,900.0
599,p_21fa7fcf997add98,Stagiaire Stage Développeur/se web FRONT ou BACK,ITALIC,Paris 10e (75),700 € - 900 € par mois,"Récapitulatif du poste Le studio ITALIC , créé...",18/09/2019,Développeur,75,stagiaire stage web front back,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1.0,1.0,1,1,1,0,1,0,0,0,0,1,0,0,recapitulatif poste studio italic cree produit...,0,1,0,0,1,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,Paris,75,700.0,900.0,month,800.0
721,p_283edb6188ab0e56,Analyste développeur H/F,TMDS,Irigny (69),600 € - 1 000 € par mois,Stage QA Tester Pourquoi venir chez nous ? Tot...,04/10/2019,Développeur,Rhône,analyste développeur,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,1.0,0,0,0,0,0,0,0,0,0,1,0,0,stage qa tester pourquoi venir chez total arke...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,Irigny,69,600.0,1000.0,month,800.0
855,p_2f2cf27029d2954b,STAGIAIRE DEVELOPPEUR FRONT-END (H/F),LEOO,Paris (75),560 € par mois,"SOCIETE LEOO, filiale du groupe ADLPerformance...",07/09/2019,Développeur,75,stagiaire developpeur,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,2.0,1,1,1,0,1,0,0,0,0,1,0,0,societe leoo filiale groupe adlperformance lea...,0,1,0,0,1,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,Paris,75,560.0,560.0,month,560.0
895,p_31d61797124bb736,Data Analyst - Stage,Pretto,Paris (75),800 € - 1 200 € par mois,Dans une startup qui croît et se structure à t...,05/10/2019,Data,75,data analyst stage,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0,0,0,0,0,1,0,1,0,0,startup croit structure toute vitesse role pol...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Paris,75,800.0,1200.0,month,1000.0
1305,p_4adf45bdd9431ed7,"Data engineer (Oracle, SQL) / Freelance",Thiveo,Paris (75),600 € par mois,Mettre en uvre des solutions techniques sécur...,07/09/2019,Data,75,data engineer oracle sql freelance,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,1,0,mettre uvre solutions techniques securisees au...,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Paris,75,600.0,600.0,month,600.0
1377,p_4f36493edf4833b5,Stage - Développeur ASP.NET / C# (H/F),Aidimpact,Toulouse (31),580 € par mois,Développeur C# .NET Dans le cadre de notre cro...,27/09/2019,Développeur,Haute-Garonne,stage développeur,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,2.0,0,0,0,0,0,0,0,0,0,1,0,0,developpeur n

# DATA CLEANING

# Missing values

In [31]:
df_salary[df_salary["salary_period"]=="year"].shape # visualize shape rows by year

(675, 78)

In [32]:
df_salary[df_salary["salary_period"]=="month"].shape # visualize shape rows by month

(30, 78)

In [33]:
df_salary[df_salary["salary_period"]=="week"].shape # visualize shape rows by week

(39, 78)

## Manage missing values

In [34]:
df_salary.reset_index(drop=True) # reseting index

,_id,Title,Company,Location,Salary,Description,Date,Job_Search,Department_Search,clean_title,Title_data,Title_data_analyst,Title_data_scientist,Title_data_engineer,Title_data_architect,Title_ingenieur,Title_big_data,Title_developpeur,Title_dev_frontend,Title_dev_web,Title_dev_mobile,Title_dev_back_end,Title_dev_fullstack,Title_devops,Title_software_eng,Title_junior,Title_manager,Title_BI,Title_confirme,Title_lead,Title_senior,Title_consultant,Title_directeur,experience_min,experience_max,BTS,DEUG,DUT,IUT,Licence,Master,Doctorat,Ingenieur,CDI,Stage,Freelance,CDD,description_clean,Python,Java,Machine Learning,Deep Learning,Javascript,Swift,NoSQL,SQL,Agile,J2EE,Ruby,HTML_CSS,Php,Big_Data,Scala,Back_end,Git,R,PowerBI,Cloud,C_C++,Docker,ABAP,WLang,city,cp,salary_min,salary_max,salary_period,salary_mean
0,p_008d77a008590232,INGÉNIEUR DÉVELOPPEUR C++ / JAVA F/H,ALTEN,Toulouse (31),30 000 € - 45 000 € par an,Partenaire technologique de référence des plus...,08/09/2019,Développeur,Haute-Garonne,ingénieur développeur java,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,1.0,0,0,0,0,0,1,0,0,1,0,0,0,partenaire technologique reference plus grande...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,Toulouse,31,30000.0,45000.0,year,37500.0
1,p_00f2ad8db3c30b8e,Devops F/H,Kent FR,Bordeaux (33),35 000 € - 40 000 € par an,"KENT, marque de GroupAgora dédiée aux recrutem...",09/03/2019,Développeur,Gironde,devops,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2.0,2.0,0,0,0,0,0,1,0,0,0,0,0,0,kent marque groupagora dediee recrutements pro...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,Bordeaux,33,35000.0,40000.0,year,37500.0
2,p_00fa03a21ad52a71,Data Scientist H/F,QYSY,Paris (75),40 000 € - 65 000 € par an,QYSY est un cabinet de conseil et de recruteme...,29/09/2019,Data,75,data scientist,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,2.0,0,0,0,0,0,1,0,0,1,0,0,0,qysy cabinet conseil recrutementqui propose re...,1,0,1,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,Paris,75,40000.0,65000.0,year,52500.0
3,p_0130bef769eef3cc,BUSINESS DEVELOPER BTOB - DATA INTELLIGENCE (H/F),L.I.P,Lyon 3e (69),35 000 € - 60 000 € par an,69003 - Lyon-3e-Arrondissement CDI Bac+5 et pl...,07/10/2019,Data,Rhône,business developer btob data intelligence,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,5.0,0,0,0,0,0,1,0,0,1,0,0,0,lyon 3e arrondissement cdi bac+5 plus ingenieu...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,Lyon,69,35000.0,60000.0,year,47500.0
4,p_019d01b7494dc632,Référent fonctionnel et données - Data manager...,GARCIN FINANCES ET PARTICIPATIONS,Corbas (69),40 000 € - 45 000 € par an,Vous souhaitez participer à la structuration d...,10/09/2019,Data,Rhône,référent fonctionnel données data manager,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1.0,1.0,0,0,0,0,0,1,0,1,1,0,0,0,souhaitez participer structuration donnees usa...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Corbas,69,40000.0,45000.0,year,42500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,pj_f9b2d5fe629ccfb3,DevOps (H/F),EASY PARTNER,Paris (75),50 000 € - 55 000 € par an,### LE POSTE Afin d’accompagner sa forte croi...,08/10/2019,Développeur,75,devops,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0,0,0,1,0,0,1,0,0,0,poste afin accompagner forte croissance souci ...,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,Paris,75,50000.0,55000.0,year,52500.0
740,pj_fc87c600387c3e85,Developpeur Java Back-end,Gentis Recruitment SAS,Paris (75),50 € par heure,"Bonjour, Notre client est une startup, jeune ...",NaN,Développeur,75,developpeur java,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0,0,1,1,0,0,0,0,0,0,bonjour client startup jeune editeur logiciel ...,0,1,0,0,1,0,1,1,1,1,0,0,0,0,1,0,1,1,0,0,0,1,0,0,Paris,7

In [35]:
df_salary[["CDI","CDD","Stage","Freelance"]].sum()

CDI          417
CDD           10
Stage         22
Freelance     58
dtype: int64

In [36]:
shape_no_contract = df_salary[(df_salary.CDI==0)&(df_salary.CDD==0)&(df_salary.Stage==0)&(df_salary.Freelance==0)].shape

In [37]:
shape_no_contract

(262, 78)

In [38]:
df_salary.shape

(744, 78)

In [39]:
print("Il y a {:.2f}% de lignes avec type de contrat non renseigné sur le df salaire".format(shape_no_contract[0]/df_salary.shape[0]*100))

Il y a 35.22% de lignes avec type de contrat non renseigné sur le df salaire


In [40]:
df_salary[(df_salary.CDI==0)&(df_salary.CDD==0)&(df_salary.Stage==0)&(df_salary.Freelance==0)&(df_salary.salary_period=="year")].shape

(252, 78)

In [41]:
df_salary[(df_salary.CDI==0)&(df_salary.CDD==0)&(df_salary.Stage==0)&(df_salary.Freelance==0)&(df_salary.salary_period=="month")].shape

(0, 78)

In [42]:
df_salary[(df_salary.CDI==0)&(df_salary.CDD==0)&(df_salary.Stage==0)&(df_salary.Freelance==0)&(df_salary.salary_period=="week")].shape

(10, 78)

In [43]:
#df_salary.iloc[653].Description

In [44]:
#df_salary[(~df_salary.experience_min.isna())&(df_salary.cdi==0)]

## Filling cdi contracts

In [45]:
# if annual salary and no contract, then contrat = CDI

# if weekly salary and no contract, then contrat = FREELANCE

In [46]:
def fill_cdi(row):
    if (row["salary_period"]=="year") & (row["CDI"]==0) & (row["Freelance"]==0) & (row["CDD"]==0) & (row["Stage"]==0):
        row["CDI"]=1
        
    return row

In [47]:
df_salary = df_salary.apply(fill_cdi, axis=1)

In [48]:
df_salary[(df_salary.CDI==0)&(df_salary.CDD==0)&(df_salary.Stage==0)&(df_salary.Freelance==0)&(df_salary.salary_period=="year")].shape

(0, 78)

## Filling freelance contracts

In [49]:
def fill_freelance(row):
    if (row["salary_period"]=="week") & (row["CDI"]==0) & (row["Freelance"]==0) & (row["CDD"]==0) & (row["Stage"]==0):
        row["Freelance"]=1
        
    return row

In [50]:
df_salary = df_salary.apply(fill_freelance, axis=1)

In [51]:
df_salary[(df_salary.CDI==0)&(df_salary.CDD==0)&(df_salary.Stage==0)&(df_salary.Freelance==0)&(df_salary.salary_period=="week")].shape

(0, 78)

# Making df salary with contrats and experience only

In [52]:
df_salary.experience_min.describe()

count    425.000000
mean       2.898824
std        1.838176
min        1.000000
25%        2.000000
50%        2.000000
75%        3.000000
max       13.000000
Name: experience_min, dtype: float64

In [53]:
df_salary.experience_max.describe()

count    425.000000
mean       3.435294
std        2.341419
min        1.000000
25%        2.000000
50%        3.000000
75%        5.000000
max       15.000000
Name: experience_max, dtype: float64

In [54]:
df_exp = df_salary[~df_salary.experience_min.isna()] # drop rows with no experience of df_salary
df_exp

,_id,Title,Company,Location,Salary,Description,Date,Job_Search,Department_Search,clean_title,Title_data,Title_data_analyst,Title_data_scientist,Title_data_engineer,Title_data_architect,Title_ingenieur,Title_big_data,Title_developpeur,Title_dev_frontend,Title_dev_web,Title_dev_mobile,Title_dev_back_end,Title_dev_fullstack,Title_devops,Title_software_eng,Title_junior,Title_manager,Title_BI,Title_confirme,Title_lead,Title_senior,Title_consultant,Title_directeur,experience_min,experience_max,BTS,DEUG,DUT,IUT,Licence,Master,Doctorat,Ingenieur,CDI,Stage,Freelance,CDD,description_clean,Python,Java,Machine Learning,Deep Learning,Javascript,Swift,NoSQL,SQL,Agile,J2EE,Ruby,HTML_CSS,Php,Big_Data,Scala,Back_end,Git,R,PowerBI,Cloud,C_C++,Docker,ABAP,WLang,city,cp,salary_min,salary_max,salary_period,salary_mean
8,p_008d77a008590232,INGÉNIEUR DÉVELOPPEUR C++ / JAVA F/H,ALTEN,Toulouse (31),30 000 € - 45 000 € par an,Partenaire technologique de référence des plus...,08/09/2019,Développeur,Haute-Garonne,ingénieur développeur java,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,1.0,0,0,0,0,0,1,0,0,1,0,0,0,partenaire technologique reference plus grande...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,Toulouse,31,30000.0,45000.0,year,37500.0
15,p_00f2ad8db3c30b8e,Devops F/H,Kent FR,Bordeaux (33),35 000 € - 40 000 € par an,"KENT, marque de GroupAgora dédiée aux recrutem...",09/03/2019,Développeur,Gironde,devops,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2.0,2.0,0,0,0,0,0,1,0,0,1,0,0,0,kent marque groupagora dediee recrutements pro...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,Bordeaux,33,35000.0,40000.0,year,37500.0
17,p_00fa03a21ad52a71,Data Scientist H/F,QYSY,Paris (75),40 000 € - 65 000 € par an,QYSY est un cabinet de conseil et de recruteme...,29/09/2019,Data,75,data scientist,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,2.0,0,0,0,0,0,1,0,0,1,0,0,0,qysy cabinet conseil recrutementqui propose re...,1,0,1,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,Paris,75,40000.0,65000.0,year,52500.0
19,p_0130bef769eef3cc,BUSINESS DEVELOPER BTOB - DATA INTELLIGENCE (H/F),L.I.P,Lyon 3e (69),35 000 € - 60 000 € par an,69003 - Lyon-3e-Arrondissement CDI Bac+5 et pl...,07/10/2019,Data,Rhône,business developer btob data intelligence,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,5.0,0,0,0,0,0,1,0,0,1,0,0,0,lyon 3e arrondissement cdi bac+5 plus ingenieu...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,Lyon,69,35000.0,60000.0,year,47500.0
31,p_019d01b7494dc632,Référent fonctionnel et données - Data manager...,GARCIN FINANCES ET PARTICIPATIONS,Corbas (69),40 000 € - 45 000 € par an,Vous souhaitez participer à la structuration d...,10/09/2019,Data,Rhône,référent fonctionnel données data manager,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1.0,1.0,0,0,0,0,0,1,0,1,1,0,0,0,souhaitez participer structuration donnees usa...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Corbas,69,40000.0,45000.0,year,42500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4844,pj_edab0aa260675678,Centre-ville / Client final / Développeur Full...,Delia Technologies,Nantes (44),37 000 € - 40 000 € par an,Delia Technologies #LaDevOpsFamily Delia Techn...,04/10/2019,Développeur,Loire-Atlantique,client final développeur java js,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,4.0,0,0,0,0,0,1,0,0,1,0,0,0,delia technologies ladevopsfamily delia techno...,0,1,0,0,1,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,Nantes,44,37000.0,40000.0,year,38500.0
4851,pj_f1d346fc78673d6a,Développeur PHP & Intégrateur HTML5/CSS3,Éraclès,Paris 15e (75),30 000 € - 37 000 € par an,Description du poste Éraclès est une agence di...,NaN,Développeur,75,développeur php intégrateur,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,5.0,0,0,0,0,0,0

## Selecting potential features

## Features

In [55]:
df_exp.columns

Index(['_id', 'Title', 'Company', 'Location', 'Salary', 'Description', 'Date',
       'Job_Search', 'Department_Search', 'clean_title', 'Title_data',
       'Title_data_analyst', 'Title_data_scientist', 'Title_data_engineer',
       'Title_data_architect', 'Title_ingenieur', 'Title_big_data',
       'Title_developpeur', 'Title_dev_frontend', 'Title_dev_web',
       'Title_dev_mobile', 'Title_dev_back_end', 'Title_dev_fullstack',
       'Title_devops', 'Title_software_eng', 'Title_junior', 'Title_manager',
       'Title_BI', 'Title_confirme', 'Title_lead', 'Title_senior',
       'Title_consultant', 'Title_directeur', 'experience_min',
       'experience_max', 'BTS', 'DEUG', 'DUT', 'IUT', 'Licence', 'Master',
       'Doctorat', 'Ingenieur', 'CDI', 'Stage', 'Freelance', 'CDD',
       'description_clean', 'Python', 'Java', 'Machine Learning',
       'Deep Learning', 'Javascript', 'Swift', 'NoSQL', 'SQL', 'Agile', 'J2EE',
       'Ruby', 'HTML_CSS', 'Php', 'Big_Data', 'Scala', 'Back_end', 'G

In [56]:
features_and_target= [ 'Title_data',
       'Title_data_analyst', 'Title_data_scientist', 'Title_data_engineer',
       'Title_data_architect', 'Title_ingenieur', 'Title_big_data',
       'Title_developpeur', 'Title_dev_frontend', 'Title_dev_web',
       'Title_dev_mobile', 'Title_dev_back_end', 'Title_dev_fullstack',
       'Title_devops', 'Title_software_eng', 'Title_junior', 'Title_manager',
       'Title_BI', 'Title_confirme', 'Title_lead', 'Title_senior',
       'Title_consultant', 'Title_directeur', 'experience_min',
       'experience_max', 'BTS', 'DEUG', 'DUT', 'IUT', 'Licence', 'Master',
       'Doctorat', 'Ingenieur', 'CDI', 'Stage', 'Freelance', 'CDD','Python', 'Java', 'Machine Learning',
       'Deep Learning', 'Javascript', 'Swift', 'NoSQL', 'SQL', 'Agile', 'J2EE',
       'Ruby', 'HTML_CSS', 'Php', 'Big_Data', 'Scala', 'Back_end', 'Git', 'R',
       'PowerBI', 'Cloud', 'C_C++', 'Docker', 'ABAP', 'WLang', 'city','Department_Search',
       'salary_min', 'salary_max', 'salary_period', 'salary_mean']

In [57]:
df_final = df_exp[features_and_target]

In [58]:
df_final.reset_index(drop=True)

,Title_data,Title_data_analyst,Title_data_scientist,Title_data_engineer,Title_data_architect,Title_ingenieur,Title_big_data,Title_developpeur,Title_dev_frontend,Title_dev_web,Title_dev_mobile,Title_dev_back_end,Title_dev_fullstack,Title_devops,Title_software_eng,Title_junior,Title_manager,Title_BI,Title_confirme,Title_lead,Title_senior,Title_consultant,Title_directeur,experience_min,experience_max,BTS,DEUG,DUT,IUT,Licence,Master,Doctorat,Ingenieur,CDI,Stage,Freelance,CDD,Python,Java,Machine Learning,Deep Learning,Javascript,Swift,NoSQL,SQL,Agile,J2EE,Ruby,HTML_CSS,Php,Big_Data,Scala,Back_end,Git,R,PowerBI,Cloud,C_C++,Docker,ABAP,WLang,city,Department_Search,salary_min,salary_max,salary_period,salary_mean
0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,1.0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,Toulouse,Haute-Garonne,30000.0,45000.0,year,37500.0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2.0,2.0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,Bordeaux,Gironde,35000.0,40000.0,year,37500.0
2,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,2.0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,Paris,75,40000.0,65000.0,year,52500.0
3,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,5.0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,Lyon,Rhône,35000.0,60000.0,year,47500.0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1.0,1.0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Corbas,Rhône,40000.0,45000.0,year,42500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,4.0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,Nantes,Loire-Atlantique,37000.0,40000.0,year,38500.0
421,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,5.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,Paris,75,30000.0,37000.0,year,33500.0
422,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,5.0,5.0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,Villeurbanne,Rhône,1750.0,1750.0,week,1750.0
423,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,3.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,Loire-Atlantique,Loire-Atlantique,22000.0,28000.0,year,25000.0


In [59]:
df_final.to_csv("df_final.csv", index=False)

x
-  experiences
-  contract
-  competences & keys words
-  qualifications
-  departement_search, city
-  Title key words

y
- salaire min, salaire mean

In [71]:
df_nosalary=df_original[(df_original.Salary.isna())&(~df_original.experience_min.isna())]

In [73]:
df_nosalary_contracts=df_nosalary[(df_original.CDI==1) | (df_original.CDD==1) |(df_original.Freelance==1) | (df_original.Stage==1)]

/Users/Ivan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [74]:
df_nosalary_contracts

,_id,Title,Company,Location,Salary,Description,Date,Job_Search,Department_Search,clean_title,Title_data,Title_data_analyst,Title_data_scientist,Title_data_engineer,Title_data_architect,Title_ingenieur,Title_big_data,Title_developpeur,Title_dev_frontend,Title_dev_web,Title_dev_mobile,Title_dev_back_end,Title_dev_fullstack,Title_devops,Title_software_eng,Title_junior,Title_manager,Title_BI,Title_confirme,Title_lead,Title_senior,Title_consultant,Title_directeur,experience_min,experience_max,BTS,DEUG,DUT,IUT,Licence,Master,Doctorat,Ingenieur,CDI,Stage,Freelance,CDD,description_clean,Python,Java,Machine Learning,Deep Learning,Javascript,Swift,NoSQL,SQL,Agile,J2EE,Ruby,HTML_CSS,Php,Big_Data,Scala,Back_end,Git,R,PowerBI,Cloud,C_C++,Docker,ABAP,WLang,city,cp
20,p_014e09f738780a82,DATA MANAGER EXPERIMENTE,Groupe IT&M,Paris (75),NaN,Offre publiée le 17/06/2019 Poste : DATA MANAG...,08/05/2019,Data,75,data manager experimente,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,4.0,4.0,0,0,0,0,1,1,0,0,1,0,0,0,offre publiee poste data manager experimente t...,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Paris,75
32,p_01a2edef7a3abe24,"Ingénieur Développeur Web et Logiciel, débutan...",ALTEN SO,Toulouse (31),NaN,La division TECHNOLOGIE d'ALTEN intervient aup...,05/10/2019,Développeur,Haute-Garonne,ingénieur développeur web logiciel débutant,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,6.0,0,0,0,0,0,1,0,0,1,0,0,0,division technologie alten intervient aupres g...,0,1,0,0,1,0,0,1,1,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,Toulouse,31
38,p_01cfe55368d233b5,DÉVELOPPEUR C#. NET (F/H),Expectra.fr,Bordeaux (33),NaN,"Expectra, leader en France du recrutement en C...",27/09/2019,Développeur,Gironde,développeur net,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,2.0,0,1,0,0,1,1,0,0,1,0,0,0,expectra leader france recrutement cdi cadres ...,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,Bordeaux,33
51,p_022dbde41ad44852,Développeur C++/Qt (QML) Windows Expériementé ...,STORMSHIELD,Lyon (69),NaN,Le poste : Développeur C++ / Qt (QML) Windows ...,07/09/2019,Data,Rhône,développeur qml windows expériementé stormshie...,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10.0,10.0,0,0,0,0,0,1,0,0,1,0,0,0,poste developpeur c++ qt qml windows travaille...,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,Lyon,69
67,p_03538e847e83ca05,Tech Lead Big Data Hadoop (H/F),ACTIVUS Group,Paris (75),NaN,"Pour renforcer notre Pôle Big Data, nous recru...",20/09/2019,Data,75,tech lead big data hadoop,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,3.0,3.0,0,0,0,0,0,1,0,0,1,0,0,0,renforcer pole big data recrutons tech lead bi...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,Paris,75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4800,pj_d79cced692d8e12d,DATA OFFICER H/F,Kelly,Paris (75),NaN,Lieu : Paris 13ème Type de contrat : CDD Salai...,NaN,Data,75,data officer,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,2.0,0,0,0,0,0,0,0,0,0,0,0,1,lieu paris eme type contrat cdd salaire k fonc...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Paris,75
4827,pj_e4799bb5625005a4,Développeur Web,Rex-Rotary,Lyon (69),NaN,Rex-Rota-quoi ? Vous ne nous connaissez pas ? ...,NaN,Développeur,Rhône,développeur web,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,3.0,0,0,0,0,0,0,0,0,1,0,0,0,rex rota quoi connaissez probleme lorsque lu q...,0,1,0,0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,Lyon,69
4848,pj_f0369c029d30e0cb,CDI – Analyste Développeur confirmé – DataStag...,BPCE SA,Paris (75),NaN,Concrétiser les besoins des métiers à travers ...,NaN,Développeur,75,cdi analyste développeur confirmé datastage,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,5.0,5.0,0,0,0,0,1,0,0,0,0,1,0,0,concretiser besoins metiers travers solution i...,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,

In [76]:
df_nosalary_contracts[df_nosalary_contracts.Stage==1].shape

(22, 74)

In [78]:
df_nosalary_contracts[df_nosalary_contracts.Freelance==1].shape

(46, 74)

In [80]:
df_original[df_original.experience_min.isna()].shape

(2866, 74)

In [108]:
features= [ 'Title_data',
       'Title_data_analyst', 'Title_data_scientist', 'Title_data_engineer',
       'Title_data_architect', 'Title_ingenieur', 'Title_big_data',
       'Title_developpeur', 'Title_dev_frontend', 'Title_dev_web',
       'Title_dev_mobile', 'Title_dev_back_end', 'Title_dev_fullstack',
       'Title_devops', 'Title_software_eng', 'Title_junior', 'Title_manager',
       'Title_BI', 'Title_confirme', 'Title_lead', 'Title_senior',
       'Title_consultant', 'Title_directeur', 'experience_min', 'BTS', 'DEUG', 'DUT', 'Licence', 'Master',
       'Doctorat', 'Ingenieur', 'CDI', 'Stage', 'Freelance', 'CDD','Python', 'Java', 'Machine Learning',
       'Deep Learning', 'Javascript', 'Swift', 'NoSQL', 'SQL', 'Agile', 'J2EE',
       'Ruby', 'HTML_CSS', 'Php', 'Big_Data', 'Scala', 'Back_end', 'Git', 'R',
       'PowerBI', 'Cloud', 'C_C++', 'Docker', 'ABAP', 'WLang', 'city','Department_Search']

In [109]:
df_to_predict=df_nosalary_contracts[features]

In [110]:
df_to_predict=df_to_predict.reset_index(drop=True)

In [111]:
df_to_predict

,Title_data,Title_data_analyst,Title_data_scientist,Title_data_engineer,Title_data_architect,Title_ingenieur,Title_big_data,Title_developpeur,Title_dev_frontend,Title_dev_web,Title_dev_mobile,Title_dev_back_end,Title_dev_fullstack,Title_devops,Title_software_eng,Title_junior,Title_manager,Title_BI,Title_confirme,Title_lead,Title_senior,Title_consultant,Title_directeur,experience_min,BTS,DEUG,DUT,Licence,Master,Doctorat,Ingenieur,CDI,Stage,Freelance,CDD,Python,Java,Machine Learning,Deep Learning,Javascript,Swift,NoSQL,SQL,Agile,J2EE,Ruby,HTML_CSS,Php,Big_Data,Scala,Back_end,Git,R,PowerBI,Cloud,C_C++,Docker,ABAP,WLang,city,Department_Search
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,4.0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Paris,75
1,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,1,1,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,Toulouse,Haute-Garonne
2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,0,1,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,Bordeaux,Gironde
3,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10.0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,Lyon,Rhône
4,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,3.0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,Paris,75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
584,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Paris,75
585,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,Lyon,Rhône
586,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,5.0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,Paris,75
587,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,Paris,75


In [112]:
df_to_predict.to_csv("to_predict.csv", index=False)